In [4]:
import pandas as pd
# import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler

### Preprocessing Temp and Rainfall Data

In [5]:
Temp_data = pd.read_csv('Temperature_Data_Export.csv')

In [6]:
Temp_data

,system:index,date,temperature,.geo
0,19800101,1980-01-01,299.401581,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,19800102,1980-01-02,298.765991,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,19800103,1980-01-03,298.795135,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,19800104,1980-01-04,299.181213,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,19800105,1980-01-05,299.083893,"{""type"":""MultiPoint"",""coordinates"":[]}"
...,...,...,...,...
14796,20200705,2020-07-05,301.647858,"{""type"":""MultiPoint"",""coordinates"":[]}"
14797,20200706,2020-07-06,300.987274,"{""type"":""MultiPoint"",""coordinates"":[]}"
14798,20200707,2020-07-07,301.741394,"{""type"":""MultiPoint"",""coordinates"":[]}"
14799,20200708,2020-07-08,301.607697,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [7]:
rainfall_data = pd.read_csv('Rainfall_Data_Export.csv')

In [8]:
rainfall_data

,system:index,date,rainfall,.geo
0,19800101,1980-01-01,0.000313,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,19800102,1980-01-02,0.000000,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,19800103,1980-01-03,0.000000,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,19800104,1980-01-04,0.000420,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,19800105,1980-01-05,0.000914,"{""type"":""MultiPoint"",""coordinates"":[]}"
...,...,...,...,...
14796,20200705,2020-07-05,0.001171,"{""type"":""MultiPoint"",""coordinates"":[]}"
14797,20200706,2020-07-06,0.010462,"{""type"":""MultiPoint"",""coordinates"":[]}"
14798,20200707,2020-07-07,0.001211,"{""type"":""MultiPoint"",""coordinates"":[]}"
14799,20200708,2020-07-08,0.001052,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [9]:
#merge rainfall and temp data

temperature_data = pd.read_csv('Temperature_Data_Export.csv')  # File with date and temperature
rainfall_data = pd.read_csv('Rainfall_Data_Export.csv')  # File with date and rainfall

# Ensure both datasets have 'date' in the same format
temperature_data['date'] = pd.to_datetime(temperature_data['date'])
rainfall_data['date'] = pd.to_datetime(rainfall_data['date'])

# Merge the two datasets on the 'date' column
temp_rainfall_data = pd.merge(temperature_data, rainfall_data, on='date', how='inner')

In [10]:
temp_rainfall_data

,system:index_x,date,temperature,.geo_x,system:index_y,rainfall,.geo_y
0,19800101,1980-01-01,299.401581,"{""type"":""MultiPoint"",""coordinates"":[]}",19800101,0.000313,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,19800102,1980-01-02,298.765991,"{""type"":""MultiPoint"",""coordinates"":[]}",19800102,0.000000,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,19800103,1980-01-03,298.795135,"{""type"":""MultiPoint"",""coordinates"":[]}",19800103,0.000000,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,19800104,1980-01-04,299.181213,"{""type"":""MultiPoint"",""coordinates"":[]}",19800104,0.000420,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,19800105,1980-01-05,299.083893,"{""type"":""MultiPoint"",""coordinates"":[]}",19800105,0.000914,"{""type"":""MultiPoint"",""coordinates"":[]}"
...,...,...,...,...,...,...,...
14796,20200705,2020-07-05,301.647858,"{""type"":""MultiPoint"",""coordinates"":[]}",20200705,0.001171,"{""type"":""MultiPoint"",""coordinates"":[]}"
14797,20200706,2020-07-06,300.987274,"{""type"":""MultiPoint"",""coordinates"":[]}",20200706,0.010462,"{""type"":""MultiPoint"",""coordinates"":[]}"
14798,20200707,2020-07-07,301.741394,"{""type"":""MultiPoint"",""coordinates"":[]}",20200707,0.001211,"{""type"":""MultiPoint"",""coordinates"":[]}"
14799,20200708,2020-07-08,301.607697,"{""type"":""MultiPoint"",""coordinates"":[]}",20200708,0.001052,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [11]:
#remove unnecessary columns
temp_rainfall_data = temp_rainfall_data.drop(['.geo_x', '.geo_y', 'system:index_y', 'system:index_x'], axis=1)

In [12]:
temp_rainfall_data

,date,temperature,rainfall
0,1980-01-01,299.401581,0.000313
1,1980-01-02,298.765991,0.000000
2,1980-01-03,298.795135,0.000000
3,1980-01-04,299.181213,0.000420
4,1980-01-05,299.083893,0.000914
...,...,...,...
14796,2020-07-05,301.647858,0.001171
14797,2020-07-06,300.987274,0.010462
14798,2020-07-07,301.741394,0.001211
14799,2020-07-08,301.607697,0.001052


In [13]:
temp_rainfall_data['date'] = pd.to_datetime(temp_rainfall_data['date']).dt.to_period('M')

In [14]:
temp_rainfall_data

,date,temperature,rainfall
0,1980-01,299.401581,0.000313
1,1980-01,298.765991,0.000000
2,1980-01,298.795135,0.000000
3,1980-01,299.181213,0.000420
4,1980-01,299.083893,0.000914
...,...,...,...
14796,2020-07,301.647858,0.001171
14797,2020-07,300.987274,0.010462
14798,2020-07,301.741394,0.001211
14799,2020-07,301.607697,0.001052


### Preprocessing Mender Migration Data

In [15]:
#Adjuest target data set
mender_dispalcement = pd.read_csv('meander_displacement.csv')

In [16]:
mender_dispalcement

,image,c1,c1.1,c1.2,c2,c2.1,c2.2,c3,c3.1,c3.2,...,c5.2,c6,c6.1,c6.2,c7,c7.1,c7.2,c8,c8.1,c8.2
0,name,x,y,dist,x,y,dist,x,y,dist,...,dist,x,y,dist,x,y,dist,x,y,dist
1,2016_12_03.png,294.4,235.0,6.664082832618455,311.5,236.0,7.738862965578342,229.5,313.0,3.2,...,26.143067914841215,352.9,356.4,34.1528915320504,366.9,430.4,173.11013835128202,385.9,474.4,190.2660505712987
2,2017_04_26.png,291.5,236.0,7.0,311.5,236.0,7.738862965578342,230.4,312.0,2.3769728648009427,...,26.485090145211892,352.9,354.5,34.1,358.9,327.5,176.05854140029675,380.9,467.4,191.45383255500528
3,2017_02_05.png,291.5,236.0,7.0,314.8,237.0,8.0,229.5,313.0,3.2,...,26.104788832702706,353.9,358.4,33.32896638061253,359.9,327.5,175.08971985813443,385.9,475.4,190.75106814904078
4,2016_08_13.png,291.5,236.0,7.0,313.5,237.0,8.104936767180853,231.4,312.0,2.906888370749727,...,26.673207531153803,352.9,357.4,34.22309161954834,357.9,331.5,176.08750665507193,385.9,476.4,191.2400847102929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,2023_04_11.png,291.5,235.0,6.0,311.5,236.0,7.738862965578342,229.5,319.0,9.2,...,27.104612153653854,353.9,356.4,33.15448687583628,353.9,357.5,176.11817055602185,385.9,476.4,191.2400847102929
180,2023_11_05.png,291.5,228.9,0.1,310.9,229.0,3.9,229.5,309.8,0.0,...,26.673207531153803,369.5,374.0,26.201145013147805,395.9,401.4,137.01503567127222,395.9,401.4,157.73639402496812
181,2024_04_06.png,165.5,1121.0,900.8551492887188,165.5,1121.0,904.4083646229726,165.5,1121.0,813.7207383371767,...,829.1340362088629,165.5,1121.0,797.8624568182163,165.5,1121.0,833.6642009826259,165.5,1121.0,833.757308813542
182,2024_11_07.png,291.5,235.0,6.0,312.5,236.0,7.368174807915459,233.4,319.0,9.992497185388645,...,26.143067914841215,353.9,338.5,36.76424893833682,366.9,414.4,168.29236465151945,366.9,414.4,188.32145390262895


### Analysis Dataset